## Classificador Automático de Sentimento com Vários Níveis de Relevância

Para implementar o programa, os tweets foram classificados em cinco níveis diferentes, e não mais somente em relevantes ou irrelevantes. Analogamente à primeira classificação, o algoritmo utilizado foi o de Naive Bayes, de acordo com cinco níveis: muito irrelevante(0); irrelevante(1); neutro(2); relevante(3) e muito relevante(4).

In [1]:
#Primeiramente importamos para poder realizar o código
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import seaborn as sns
from emoji import UNICODE_EMOJI
import matplotlib.pyplot as plt
from operator import sub

In [2]:
# leitura dos arquivos em excel
nike = pd.read_excel('1537437860647_tweets_nike_201809042210.xlsx', sheet_name = 0)
nike_treinamento = nike["Treinamento"]
nike.head(2)

,Treinamento,Valor_(R(1)_I(0)),Níveis
0,save big with joyhunt-up to 80% off rpp on gre...,1,2
1,rt @virgilableaux: hi. former us navy sailor h...,1,4


In [3]:
import string
pontu = string.punctuation

#criação de lista para tirar os tabs e os enters
pont =["\n", "\t"]

#simplificação para pegar o dataframde dos tweets
tweet = nike["Treinamento"]

### limpando os tweets
tweets_limpos = []

for frase in tweet:
    x = ""
    for m in frase:
        if m in UNICODE_EMOJI:
            x = x + " " + m + " "
        elif m in pont:
            x += " "
        elif m not in pontu:
            x += m
    tweets_limpos.append(x)

# criando um dataframe com os tweets limpos
nike_limpinho = pd.DataFrame()
nike_limpinho["tweets"]= tweets_limpos
nike_limpinho["Níveis"] = nike["Níveis"]

nike_limpinho.head()

,tweets,Níveis
0,save big with joyhuntup to 80 off rpp on great...,2
1,rt virgilableaux hi former us navy sailor here...,4
2,boycottnike i think a great big thank you nike...,4
3,rt thedonholly if you plan on boycotting nike ...,3
4,rt thedonholly if you plan on boycotting nike ...,3


In [4]:
## limpar os nomes:
    # tirar tudo que é irrelevante para minha pesquisa e para descobrir a probabilidade como: @,
    # palavras com poucas letras, #, link(site).

tweet_limpos =[]
y = " "
for tweet in nike_limpinho["tweets"]:
    limpao = [] 
    splitei = tweet.split(" ")
    for palavra in splitei:
        if palavra in UNICODE_EMOJI:
            limpao.append(palavra)
        elif len(palavra)> 2 and palavra[0]!="@" and palavra[0]!="#" and palavra[:4] !="http":
            limpao.append(palavra)
    tweet_limpos.append(y.join(limpao))
    
nike_limpao = pd.DataFrame()
nike_limpao["tweets"]= tweet_limpos
nike_limpao["Níveis"] = nike["Níveis"]

nike_limpao.head()

,tweets,Níveis
0,save big with joyhuntup off rpp great brands s...,2
1,virgilableaux former navy sailor here wonderin...,4
2,boycottnike think great big thank you nike fro...,4
3,thedonholly you plan boycotting nike will disp...,3
4,thedonholly you plan boycotting nike will disp...,3


In [5]:
palavras = []

#adicionando todas palavras que foram recolhidas pelo tweet na lista
for e in nike_limpao["tweets"]: 
    y = e.split()
    for m in y:
        if m not in palavras: 
            palavras.append(m)

#adicionar a quantidade de vezes que apareceu tweets relevantes e irrelevantes
MIrrel = 0
Irrel = 0
Neutro = 0
Rel=0
MRel = 0
for i in range(len(nike_limpao)):
    linha = nike_limpao["tweets"][i].split(" ")
    for m in linha:
        if nike_limpao["Níveis"][i] == 0:
            MIrrel +=1
        elif nike_limpao["Níveis"][i]== 1:
            Irrel+=1
        elif nike_limpao["Níveis"][i]== 2:
            Neutro+=1
        elif nike_limpao["Níveis"][i]== 3:
            Rel+=1
        else:
            MRel +=1

In [6]:
#calculando a frequência com a qual cada palavra aparece em cada nível de classificação
freq_MR = {}
freq_R = {}
freq_MI = {}
freq_I = {}
freq_N = {}

for palavra in palavras:
    freq_MR[palavra] = 1
    freq_R[palavra] = 1
    freq_MI[palavra] = 1
    freq_I[palavra] = 1
    freq_N[palavra] = 1

for i in range(len(nike_limpao)):
    palavra = nike_limpao["tweets"][i].split(" ")
    for m in palavra:
        if nike_limpao["Níveis"][i]== 0:
            freq_MI[m]+=1
        elif nike_limpao["Níveis"][i]== 1:
            freq_I[m]+=1
        elif nike_limpao["Níveis"][i]== 2:
            freq_N[m]+=1
        elif nike_limpao["Níveis"][i]== 3:
            freq_R[m]+=1
        else:
            freq_MR[m]+=1
         

In [7]:
#probabilidade de ser relevante dada cada palavra

prob_mi ={}
prob_i ={}
prob_n ={}
prob_r = {}
prob_mr ={}

for palavra in palavras:
    prob_mi[palavra]= freq_MI[palavra]/(len(palavras)+MIrrel)
    prob_i[palavra]= freq_I[palavra]/(len(palavras)+Irrel)
    prob_n[palavra]= freq_N[palavra]/(len(palavras)+Neutro)
    prob_r[palavra]= freq_R[palavra]/(len(palavras)+Rel)
    prob_mr[palavra]= freq_MR[palavra]/(len(palavras)+MRel)

In [8]:
# calculando a probabilidade de ser relevante dado cada tweet

m_irrelevante = 0
irrelevante = 0
neutro=0
relevante = 0
m_relevante =0

for p in nike_limpao["Níveis"]:
    if p ==0:
        m_irrelevante+=1
    elif p ==1:
        irrelevante+=1
    elif p ==2:
        neutro+=1
    elif p ==3:
        relevante+=1
    else:
        m_relevante +=1
        
pMI = m_irrelevante/len(nike_limpao["Níveis"])
pI = irrelevante/len(nike_limpao["Níveis"])
pN = neutro/len(nike_limpao["Níveis"])
pR = relevante/len(nike_limpao["Níveis"])
pMR = m_relevante/len(nike_limpao["Níveis"])

In [9]:
# lendo a segunda página para ir ao teste
teste = pd.read_excel('1537437860647_tweets_nike_201809042210.xlsx', sheet_name = 1)

In [10]:
#fazendo a mesma limpeza de frases/tweets

#criação de lista para tirar os tabs e os enters
pont =["\n", "\t"]

#simplificação para pegar o dataframde dos tweets
tweet = teste["Teste"]

### limpando os tweets
tweets_limpos = []

for frase in tweet:
    x = ""
    for m in frase:
        if m in UNICODE_EMOJI:
            x = x + " " + m + " "
        elif m in pont:
            x += " "
        elif m not in pontu:
            x += m
    tweets_limpos.append(x)

# criando um dataframe com os tweets limpos
nike_limpinho = pd.DataFrame()
nike_limpinho["tweets"]= tweets_limpos
nike_limpinho["Níveis"] = teste["Níveis"]

nike_limpinho.head()

,tweets,Níveis
0,rt jamiesundays nike looking at the white peop...,4
1,trump weighs in on nikes kaepernick deal i thi...,3
2,rt rafaelshimunov im finally cutting out the n...,3
3,dear nike i am not going to boycott you becaus...,3
4,rt lindsaytuten anyone who doesn’t want their ...,3


In [11]:
# código para fazer o programa classificar os tweets a partir da probabilidade

tweet_limpos =[]
y = " "
for tweet in nike_limpinho["tweets"]:
    limpao = [] 
    splitei = tweet.split(" ")
    for palavra in splitei:
        if palavra in UNICODE_EMOJI:
            limpao.append(palavra)
        elif len(palavra)> 2 and palavra[0]!="@" and palavra[0]!="#" and palavra[:4] !="http":
            limpao.append(palavra)
    tweet_limpos.append(y.join(limpao))
    
nike_limpao = pd.DataFrame()
nike_limpao["tweets"]= tweet_limpos
nike_limpao["Níveis"] = teste["Níveis"]

nike_limpao.head()

,tweets,Níveis
0,jamiesundays nike looking the white people des...,4
1,trump weighs nikes kaepernick deal think its t...,3
2,rafaelshimunov finally cutting out the nike lo...,3
3,dear nike not going boycott you because just s...,3
4,lindsaytuten anyone who doesn’t want their nik...,3


In [12]:
#fazendo P(frase|rel) * P(Rel)
chute = []

for frase in nike_limpao["tweets"]:
    pMI_t = 1
    pI_t = 1
    pN_t = 1
    pR_t= 1
    pMR_t = 1
    y = frase.split(" ")
    for palavra in y:
        
        if palavra in prob_mi:
            pMI_t *= prob_mi[palavra]
        else:
            pMI_t *= 1/ (len(palavras)+MIrrel)
        
        if palavra in prob_i:
            pI_t *= prob_i[palavra]
        else:
            pI_t *= 1/ (len(palavras)+Irrel)
            
        if palavra in prob_n:
            pN_t *= prob_n[palavra]
        else:
            pN_t *= 1/ (len(palavras)+Neutro)
            
        if palavra in prob_r:
            pR_t*= prob_r[palavra]
        else:
            pR_t *= 1/(len(palavra)+Rel)
            
        if palavra in prob_mr:
            pMR_t *= prob_mr[palavra]
        else:
            pMR_t *= 1/ (len(palavras)+MRel)

  
    PMIR = (pMI*pMI_t)
    PIR =(pI*pI_t)
    PN = (pN*pN_t)
    PR =( pR*pR_t)
    PMR = (pMR*pMR_t)
    
    if PMR >= PR and PMR>=PN and PMR>=PIR and PMR>=PMIR:
        chute.append(4)
        
    elif PR>=PN and PR>=PIR and PR>=PMIR:
        chute.append(3)
        
    elif PN>=PIR and PN>=PMIR:
        chute.append(2)
        
    elif PIR>=PMIR:
        chute.append(1)
        
    else:
        chute.append(0)
        
        
# adicionando essa coluna ao nosso dataframe

nike_limpao["Chute"] = chute
nike_limpao.head()

,tweets,Níveis,Chute
0,jamiesundays nike looking the white people des...,4,3
1,trump weighs nikes kaepernick deal think its t...,3,4
2,rafaelshimunov finally cutting out the nike lo...,3,4
3,dear nike not going boycott you because just s...,3,3
4,lindsaytuten anyone who doesn’t want their nik...,3,3


In [19]:
# comparando a classificação do teste e do treinamento

zero = nike_limpao.loc[(nike_limpao["Níveis"]==0)&(nike_limpao["Chute"]==0), ["Níveis", "Chute"]]
zeroC = nike_limpao.loc[(nike_limpao["Níveis"]!=0)&(nike_limpao["Chute"]==0), ["Níveis", "Chute"]]

um = nike_limpao.loc[(nike_limpao["Níveis"]==1)&(nike_limpao["Chute"]==1), ["Níveis", "Chute"]]
umC = nike_limpao.loc[(nike_limpao["Níveis"]!=1)&(nike_limpao["Chute"]==1), ["Níveis", "Chute"]]

dois = nike_limpao.loc[(nike_limpao["Níveis"]==2)&(nike_limpao["Chute"]==2), ["Níveis", "Chute"]]
doisC = nike_limpao.loc[(nike_limpao["Níveis"]!=2)&(nike_limpao["Chute"]==2), ["Níveis", "Chute"]]

tres = nike_limpao.loc[(nike_limpao["Níveis"]==3)&(nike_limpao["Chute"]==3), ["Níveis", "Chute"]]
tresC= nike_limpao.loc[(nike_limpao["Níveis"]!=3)&(nike_limpao["Chute"]==3), ["Níveis", "Chute"]]

quatro= nike_limpao.loc[(nike_limpao["Níveis"]==4)&(nike_limpao["Chute"]==4), ["Níveis", "Chute"]]
quatroC = nike_limpao.loc[(nike_limpao["Níveis"]!=4)&(nike_limpao["Chute"]==4), ["Níveis", "Chute"]]

In [14]:
#fazendo uma matriz de confusão

from sklearn.metrics import confusion_matrix
print(confusion_matrix(nike_limpao["Níveis"], nike_limpao["Chute"]))

[[ 0  0  0  4  0]
 [ 0  0  0  2  0]
 [ 0  0  2  5  0]
 [ 0  0 11 91 47]
 [ 0  0  0 27 11]]


In [15]:
print(pd.crosstab(nike_limpao["Níveis"], nike_limpao["Chute"], rownames=["Real"], colnames=["Predito"], margins=True, normalize=True))

Predito      2      3      4    All
Real                               
0        0.000  0.020  0.000  0.020
1        0.000  0.010  0.000  0.010
2        0.010  0.025  0.000  0.035
3        0.055  0.455  0.235  0.745
4        0.000  0.135  0.055  0.190
All      0.065  0.645  0.290  1.000


In [16]:
#calculando a probabilidade de positivos verdadeiros, positivos falsos, negativos verdadeiros e negativos falsos para cada nível

pos_ver = (len(um)/len(nike_limpao["Níveis"]))*100
pos_fal = (len(umC)/len(nike_limpao["Níveis"]))*100

neg_ver = (len(zero)/len(nike_limpao["Níveis"]))*100
neg_fal= (len(zeroC)/len(nike_limpao["Níveis"]))*100



In [20]:
#calculando a probabilidade de positivos verdadeiros, positivos falsos, negativos verdadeiros e negativos falsos totais do classificador

print("Porcentagem de positivos falsos: {0}%".format(pos_fal))
print("Porcentagem de positivos verdadeiros: {0}%".format(pos_ver))
print("Porcentagem de negativos verdadeiros: {0}%".format(neg_ver))
print("Porcentagem de negativos falsos: {0}%".format(neg_fal))

Porcentagem de positivos falsos: 0.0%
Porcentagem de positivos verdadeiros: 0.0%
Porcentagem de negativos verdadeiros: 0.0%
Porcentagem de negativos falsos: 0.0%


## Conclusão